In [1]:
import advent
advent.scrape(2024, 21)
data = advent.get_lines(21)
print(data)

['208A', '540A', '685A', '879A', '826A']


In [2]:
from typing import NamedTuple, Iterator
from advent.maze import solve_maze, solve_maze_no_tqdm

class Node(NamedTuple):
    bottom: str # the main keypad
    middle: str # The radiated keypad
    top: str    # The -40 degrees keypad
    code: str   # Code that has been entered so far
    target_code: str # Target code that must be entered

def is_target(node):
    return node.code == node.target_code


bottom_adj = {
    '0': {'^': '2', '>': 'A'},
    '1': {'>': '2', '^': '4'},
    '2': {'v': '0', '<': '1', '>': '3', '^': '5'},
    '3': {'<': '2', '^': '6', 'v': 'A'},
    '4': {'v': '1', '>': '5', '^': '7'},
    '5': {'v': '2', '<': '4', '>': '6', '^': '8'},
    '6': {'v': '3', '<': '5', '^': '9'},
    '7': {'v': '4', '>': '8'},
    '8': {'v': '5', '<': '7', '>': '9'},
    '9': {'v': '6', '<': '8'},
    'A': {'<': '0', '^': '3'}
}

topmid_adj = {
    '<': {'>': 'v'},
    '^': {'v': 'v', '>': 'A'},
    'v': {'<': '<', '^': '^', '>': '>'},
    '>': {'<': 'v', '^': 'A'},
    'A': {'<': '^', 'v': '>'}
}

In [3]:
def adjacent(node: Node) -> Iterator[tuple[Node, int]]:
    # We pressed the wrong button somewhere: this node is a dead end
    if not node.target_code.startswith(node.code): return []
    # We can press either arrow or A
    # Press arrow: top moves. we only press the moves top has available
    for dir in topmid_adj[node.top]:
        yield Node(node.bottom, node.middle, topmid_adj[node.top][dir], node.code, node.target_code), 1
    # Press A: top stays the same. Now either top was arrow or A. first: top was arrow
    if node.top != 'A':
        # Middle will move according to what node.top was
        if node.top in topmid_adj[node.middle]:
            yield Node(node.bottom, topmid_adj[node.middle][node.top], node.top, node.code, node.target_code), 1
    # Press A, and top was also A. Middle will stay the same. Now either middle was arrow or A.
    elif node.middle != 'A':
        # bottom will move according to what node.middle was
        if node.middle in bottom_adj[node.bottom]:
            yield Node(bottom_adj[node.bottom][node.middle], node.middle, node.top, node.code, node.target_code), 1
    # Top and middle were both A. that means all robots will stay the same, and the code will be updated
    else:
        yield Node(node.bottom, node.middle, node.top, node.code + node.bottom, node.target_code), 1

start = Node('A', 'A', 'A', '', '029A')
l, _, _ = solve_maze_no_tqdm(start, is_target, adjacent)
assert l == 68
    

In [4]:
result = 0
for line in data:
    start = Node('A', 'A', 'A', '', line)
    l, _, _ = solve_maze_no_tqdm(start, is_target, adjacent)
    result += int(line[:3]) * l
print(result)

224326


In [8]:
# Part 2 :D
from functools import cache

class Node2(NamedTuple):
    bottom: str
    middles: tuple[str, ...]
    code: str
    target_code: str

@cache
def apply_press(middles: tuple[str, ...], button: str) -> None|tuple[tuple[str, ...], None|str]:
    # Returns None if this button may not be pressed
    if len(middles) == 0: return (), button
    top, rest = middles[0], middles[1:]
    if button != 'A':
        if button in topmid_adj[top]:
            return (topmid_adj[top][button],) + rest, None
        else:
            return None
    else:
        # top will stay the same. for the rest, use recursion
        subresult = apply_press(rest, top)
        if subresult is not None: return (top,) + subresult[0], subresult[1]
        else: return None

def adjacent2(node: Node2) -> Iterator[tuple[Node2, int]]:
    # We pressed the wrong button somewhere: this node is a dead end
    if not node.target_code.startswith(node.code): return []
    # We can press either arrow or A
    # Press arrow: top moves. we only press the moves top has available
    for buttonpress in ['<', 'v', '>', '^', 'A']:
        newmiddles = apply_press(node.middles, buttonpress)
        if newmiddles is None: continue # This press is not allowed
        newmiddles, bottombutton = newmiddles
        if bottombutton == 'A':
            yield Node2(node.bottom, node.middles, node.code + node.bottom, node.target_code), 1
        elif bottombutton is not None:
            if bottombutton in bottom_adj[node.bottom]:
                yield Node2(bottom_adj[node.bottom][bottombutton], newmiddles, node.code, node.target_code), 1
        else:
            yield Node2(node.bottom, newmiddles, node.code, node.target_code), 1
        
middle_keypads = 6  # 2 for part 1, 25 for part 2

result = 0
for line in data:
    start = Node2('A', ('A',) * middle_keypads, '', line)
    l, _, _ = solve_maze_no_tqdm(start, is_target, adjacent2)
    result += int(line[:3]) * l
print(result)

8447256


In [ ]:
# Unfortunately this seems to get exponentially slower and is already pretty slow at just 7 keypads.
# So we have to think of something more clever...